In [ ]:
import numpy as np
import pandas as pd
import re

# This is for all bavaria, the other notebook is for munich only.

# --- 1) Read the Data ---
census_2024_df = pd.read_csv("census_data.csv", sep=",", dtype={"commune_id": str})  # keep strings as strings
n_distinct = census_2024_df.iloc[:, 0].nunique(dropna=True)
print("Number of distinct comune ids:", n_distinct)
print("In total, the population for Oberbayern, Niederbayern and Schwaben is:", census_2024_df['weight'].sum())
# mask_munich = census_df["commune_id"].astype(str).str.startswith("9162")
# total_weight = census_df.loc[mask_munich, "weight"].sum(min_count=1)  # returns NaN if none match
# print("There are ", total_weight, " people in Munich")

# sex_weights_df = census_df[mask_munich]
# age_classes_target_classes = sorted(census_df.loc[mask_munich, "age_class"].unique())
population_2045 = pd.read_csv("population_oberbayern_niederbayern_schwaben_2043.csv", sep=",", dtype={"kommunen_id_start": str})

Number of distinct comune ids: 1098
In total, the population for Oberbayern, Niederbayern and Schwaben is: 8021702


In [25]:
len(census_2024_df)

28548

In [ ]:
# Ensure types are as expected
census_2024_df["commune_id"] = census_2024_df["commune_id"].astype(str).str.strip()
census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce")

# Drop rows without an ID
census_2024_df = census_2024_df[census_2024_df["commune_id"].notna() & (census_2024_df["commune_id"] != "")]

# Build: commune_id -> total population (sum of weights)
census_2_pop = (
    census_2024_df.groupby("commune_id", dropna=False)["weight"]
             .sum(min_count=1)         # keeps NaN if all weights were NaN for an ID
             .fillna(0)                # optional: treat all-NaN as 0
             .to_dict()
)

In [ ]:
# -------------------- Helpers --------------------
def normalize_id(series: pd.Series) -> pd.Series:
    """Return 12-digit string IDs (digits only, left-padded). Drop invalid."""
    s = (series.astype(str)
               .str.strip()
               .str.replace(r"\D", "", regex=True)
               .str.zfill(12))
    return s.where(s.str.fullmatch(r"\d{12}"))

# -------------------- Inputs --------------------
# 1) population_2045: one row per 5-digit prefix
#    Must have columns: kommunen_id_start (str), Name_norm (str), population_2043_abs (int)
# population_2045 = pd.read_csv(
#     "kommunen_oberbayern_niederbayern_schwaben_2043_with_names_start_only.csv",
#     dtype={"kommunen_id_start": str}
population_2045 = population_2045[["kommunen_id_start","Name_norm","population_2043_abs"]].copy()

# 2) census input: either a dict census_2_pop or a census_df to build it
# If you already have census_2_pop, comment out the block that builds it.

try:
    census_2_pop  # dict exists
except NameError:
    # Build from census_df
    census_df = pd.read_csv("census_data.csv", dtype={"commune_id": str})
    census_df["commune_id_norm"] = normalize_id(census_df["commune_id"])
    census_df = census_df[census_df["commune_id_norm"].notna()].copy()
    census_df["weight"] = pd.to_numeric(census_df["weight"], errors="coerce").fillna(0)

    # Sum weights per commune (current population by commune)
    census_2_pop = (census_df.groupby("commune_id_norm")["weight"]
                    .sum(min_count=1).fillna(0).to_dict())

# -------------------- Build commune-level frame --------------------
# Turn census_2_pop dict into a DataFrame
commune_now = (pd.Series(census_2_pop, name="population_2024_kommune")
               .rename_axis("kommunen_id")  # 12-digit id
               .reset_index())

# Derive 5-digit prefix for each commune
commune_now["kommunen_id"] = normalize_id(commune_now["kommunen_id"])
commune_now = commune_now[commune_now["kommunen_id"].notna()].copy()
commune_now["kommunen_id_start"] = commune_now["kommunen_id"].str[:5]

# -------------------- Aggregate current pop by prefix --------------------
grp_now = (commune_now.groupby("kommunen_id_start", dropna=True)["population_2024_kommune"]
           .sum(min_count=1).rename("population_2024_gemeinde")
           .reset_index())

# Attach the group total and the PDF 2043 totals & name (by prefix)
work = (commune_now
        .merge(grp_now, on="kommunen_id_start", how="left")
        .merge(population_2045.rename(columns={"population_2043_abs":"population_2043_gemeinde",
                                               "Name_norm":"name"}),
               on="kommunen_id_start", how="left"))

# -------------------- Compute shares & scaled 2043 --------------------
# Guard against division by zero: if the group total is 0 or NaN, set share=0 and scaled=0
den = work["population_2024_gemeinde"].replace({0: np.nan})
share = work["population_2024_kommune"] / den
work["weight_of_single_kommune_among_gemeinde"] = (share.fillna(0) * 100)  # percent
work["population_2043_gemeinde"] = pd.to_numeric(work["population_2043_gemeinde"], errors="coerce").fillna(0)

work["scaled_population_2043_kommune"] = (
    work["population_2043_gemeinde"] * (work["weight_of_single_kommune_among_gemeinde"] / 100.0)
).round(0).astype(int)

# -------------------- Final columns & save --------------------
result = work[[
    "kommunen_id",                        # 12-digit
    "kommunen_id_start",                  # 5-digit prefix
    "name",                               # from the prefix row
    "population_2024_kommune",            # current pop by commune (from census_2_pop)
    "population_2024_gemeinde",           # sum over the prefix
    "weight_of_single_kommune_among_gemeinde",  # percent
    "population_2043_gemeinde",           # PDF 2043 per prefix
    "scaled_population_2043_kommune"      # allocated 2043 per commune
]].copy()

# Optional: tidy types / rounding
result["weight_of_single_kommune_among_gemeinde"] = result["weight_of_single_kommune_among_gemeinde"].round(6)

Rows: 1098
    kommunen_id kommunen_id_start                         name  \
0  091610000000             09161  Kreisfreie Stadt Ingolstadt   
1  091620000000             09162    München, Landeshauptstadt   
2  091630000000             09163   Kreisfreie Stadt Rosenheim   
3  091710111111             09171          Landkreis Altötting   
4  091710112112             09171          Landkreis Altötting   
5  091710113113             09171          Landkreis Altötting   
6  091710117117             09171          Landkreis Altötting   
7  091710118118             09171          Landkreis Altötting   
8  091710125125             09171          Landkreis Altötting   
9  091710127127             09171          Landkreis Altötting   

   population_2024_kommune  population_2024_gemeinde  \
0                   141029                    141029   
1                  1512491                   1512491   
2                    64403                     64403   
3                    13104            

In [ ]:
result_cleaned = result[["kommunen_id", "population_2024_kommune", "scaled_population_2043_kommune"]]
result_cleaned = result_cleaned.rename(
    columns={
        "population_2024_kommune": "pop_2024",
        "scaled_population_2043_kommune": "pop_2043",
    }
)

In [31]:
result_cleaned

,kommunen_id,pop_2024,pop_2043
0,091610000000,141029,150500
1,091620000000,1512491,1550800
2,091630000000,64403,66700
3,091710111111,13104,13634
4,091710112112,19364,20147
...,...,...,...
1093,097805742121,3311,3398
1094,097805742131,1029,1056
1095,097805742134,2071,2125
1096,097805745127,1516,1556


In [ ]:
import pandas as pd
import numpy as np
import re

# -------------------- Helpers --------------------
def normalize_id(series: pd.Series) -> pd.Series:
    """Return 12-digit string IDs (digits only, left-padded). Drop invalid."""
    s = (series.astype(str)
               .str.strip()
               .str.replace(r"\D", "", regex=True)
               .str.zfill(12))
    return s.where(s.str.fullmatch(r"\d{12}"))

# -------------------- Inputs --------------------
# result_cleaned must have: kommunen_id, pop_2024, pop_2043
# census_df must have: commune_id, sex, age_class, weight
# If you already have them in memory, you can skip these reads.
# result_cleaned = pd.read_csv("result_cleaned.csv", dtype={"kommunen_id": str})
# census_df = pd.read_csv("census_data.csv", dtype={"commune_id": str})

# --- Normalize IDs and clean types
result_cleaned["kommunen_id"] = normalize_id(result_cleaned["kommunen_id"])
result_cleaned = result_cleaned[result_cleaned["kommunen_id"].notna()].copy()

census_df["commune_id_norm"] = normalize_id(census_df["commune_id"])
census_df = census_df[census_df["commune_id_norm"].notna()].copy()
census_df["weight"] = pd.to_numeric(census_df["weight"], errors="coerce").fillna(0)

# -------------------- 1) Enumerate the 26 combinations --------------------
all_combos = (
    census_df[["sex", "age_class"]]
      .drop_duplicates()
      .sort_values(["sex", "age_class"])
      .reset_index(drop=True)
)
# Sanity: we expect 26 distinct combos total
n_combos = len(all_combos)
if n_combos != 26:
    print(f"WARNING: expected 26 (sex, age_class) combos, found {n_combos}. Proceeding with found combos.")

# -------------------- 2) Compute per-(commune_id, combo) weights & fractions --------------------
combo_weights = (
    census_df
      .groupby(["commune_id_norm", "sex", "age_class"], dropna=False)["weight"]
      .sum(min_count=1)
      .reset_index()
)

totals_per_commune = (
    combo_weights
      .groupby("commune_id_norm", dropna=False)["weight"]
      .sum(min_count=1)
      .rename("total_weight_commune")
      .reset_index()
)

combo_weights = combo_weights.merge(totals_per_commune, on="commune_id_norm", how="left")

# Fraction per combo (guard against division by zero)
combo_weights["fraction_2024"] = (
    combo_weights["weight"] / combo_weights["total_weight_commune"].replace({0: np.nan})
).fillna(0.0)

# -------------------- 3) Ensure every commune has all combos (fill missing with 0) --------------------
communes = result_cleaned[["kommunen_id"]].drop_duplicates().rename(columns={"kommunen_id": "commune_id_norm"})
grid = communes.assign(key=1).merge(all_combos.assign(key=1), on="key").drop(columns="key")

grid = grid.merge(
    combo_weights[["commune_id_norm", "sex", "age_class", "fraction_2024"]],
    on=["commune_id_norm", "sex", "age_class"],
    how="left"
)
grid["fraction_2024"] = grid["fraction_2024"].fillna(0.0)

# OPTIONAL EVEN-SPLIT FALLBACK:
# If a commune has total_weight_commune == 0 (i.e., no data at all), you might want to split evenly across combos:
# zero_data_communes = grid.groupby("commune_id_norm")["fraction_2024"].sum().reset_index()
# zero_data_communes = zero_data_communes[zero_data_communes["fraction_2024"] == 0]["commune_id_norm"]
# grid.loc[grid["commune_id_norm"].isin(zero_data_communes), "fraction_2024"] = 1.0 / n_combos

# -------------------- 4) Expand result_cleaned (26 rows per commune) and scale pops --------------------
expanded = (
    grid.merge(
        result_cleaned.rename(columns={"kommunen_id": "commune_id_norm"}),
        on="commune_id_norm",
        how="left"
    )
)

expanded["pop_2024_by_combo"] = expanded["pop_2024"] * expanded["fraction_2024"]
expanded["pop_2043_by_combo"] = expanded["pop_2043"] * expanded["fraction_2024"]

# -------------------- 5) Validations --------------------
# A) Fractions per commune should sum to 1 (or 0 if truly no data and no fallback)
frac_sums = expanded.groupby("commune_id_norm")["fraction_2024"].sum().reset_index()
bad_fracs = frac_sums[~np.isclose(frac_sums["fraction_2024"], 1.0, atol=1e-8) & (frac_sums["fraction_2024"] != 0)]
if not bad_fracs.empty:
    print("WARNING: Some communes have fractions not summing to 1 (and not 0). Examples:")
    print(bad_fracs.head())

# B) Check that sums of distributed pops match originals (within tolerance)
check_2024 = (
    expanded.groupby("commune_id_norm")["pop_2024_by_combo"].sum().reset_index()
    .merge(result_cleaned.rename(columns={"kommunen_id":"commune_id_norm"})[["commune_id_norm","pop_2024"]],
           on="commune_id_norm", how="left")
)
check_2024["diff_2024"] = check_2024["pop_2024_by_combo"] - check_2024["pop_2024"]
if not np.allclose(check_2024["diff_2024"], 0.0, atol=1e-6, rtol=1e-10):
    print("WARNING: pop_2024_by_combo does not perfectly sum back to pop_2024 for some communes.")

check_2043 = (
    expanded.groupby("commune_id_norm")["pop_2043_by_combo"].sum().reset_index()
    .merge(result_cleaned.rename(columns={"kommunen_id":"commune_id_norm"})[["commune_id_norm","pop_2043"]],
           on="commune_id_norm", how="left")
)
check_2043["diff_2043"] = check_2043["pop_2043_by_combo"] - check_2043["pop_2043"]
if not np.allclose(check_2043["diff_2043"], 0.0, atol=1e-6, rtol=1e-10):
    print("WARNING: pop_2043_by_combo does not perfectly sum back to pop_2043 for some communes.")

# -------------------- 6) Final tidy --------------------
result_expanded = expanded[[
    "commune_id_norm",  # == kommunen_id (12 digits)
    "sex",
    "age_class",
    "fraction_2024",
    "pop_2024_by_combo",
    "pop_2043_by_combo",
    "pop_2024",
    "pop_2043",
]].rename(columns={"commune_id_norm": "kommunen_id"})

# (Optional) Round to integers if you need person counts:
# result_expanded["pop_2024_by_combo"] = result_expanded["pop_2024_by_combo"].round(0).astype(int)
# result_expanded["pop_2043_by_combo"] = result_expanded["pop_2043_by_combo"].round(0).astype(int)

# Save if desired
# result_expanded.to_csv("result_expanded_26_combos.csv", index=False)

Rows in expanded result: 28548
    kommunen_id     sex  age_class  fraction_2024  pop_2024_by_combo  \
0  091610000000  female          0       0.015983             2254.0   
1  091610000000  female          3       0.015642             2206.0   
2  091610000000  female          6       0.019414             2738.0   
3  091610000000  female         10       0.021414             3020.0   
4  091610000000  female         15       0.012607             1778.0   
5  091610000000  female         18       0.009381             1323.0   
6  091610000000  female         20       0.026051             3674.0   
7  091610000000  female         25       0.033759             4761.0   
8  091610000000  female         30       0.074077            10447.0   
9  091610000000  female         40       0.064561             9105.0   

   pop_2043_by_combo  pop_2024  pop_2043  
0        2405.370527    141029    150500  
1        2354.147019    141029    150500  
2        2921.874224    141029    150500  
3   

In [35]:
result_expanded.head(20)

,kommunen_id,sex,age_class,fraction_2024,pop_2024_by_combo,pop_2043_by_combo,pop_2024,pop_2043
0,091610000000,female,0,0.015983,2254.0,2405.370527,141029,150500
1,091610000000,female,3,0.015642,2206.0,2354.147019,141029,150500
2,091610000000,female,6,0.019414,2738.0,2921.874224,141029,150500
3,091610000000,female,10,0.021414,3020.0,3222.812329,141029,150500
4,091610000000,female,15,0.012607,1778.0,1897.404080,141029,150500
5,091610000000,female,18,0.009381,1323.0,1411.847918,141029,150500
6,091610000000,female,20,0.026051,3674.0,3920.732615,141029,150500
7,091610000000,female,25,0.033759,4761.0,5080.731623,141029,150500
8,091610000000,female,30,0.074077,10447.0,11148.582916,141029,150500
9,091610000000,female,40,0.064561,9105.0,9716.459026,141029,150500


In [ ]:
final_result = result_expanded.rename(
    columns={
        "kommunen_id": "commune_id",
        "pop_2043_by_combo": "weight",
    }
)

In [44]:
final_result=final_result[["commune_id", "sex","age_class","weight"]]

In [51]:
final_result["commune_id"]

0        091610000000
1        091610000000
2        091610000000
3        091610000000
4        091610000000
             ...     
28543    097805745144
28544    097805745144
28545    097805745144
28546    097805745144
28547    097805745144
Name: commune_id, Length: 28548, dtype: object

In [ ]:
# final_result.to_csv("census_data_2043", index=False)

In [ ]:
final_result_2 = final_result.copy()
final_result_2["commune_id"] = final_result_2["commune_id"].astype(str)
subset = final_result_2[final_result_2["commune_id"] == "091620000000"]

In [53]:
subset

,commune_id,sex,age_class,weight
26,091620000000,female,0,24236.713078
27,091620000000,female,3,22451.616307
28,091620000000,female,6,27773.070782
29,091620000000,female,10,31242.782139
30,091620000000,female,15,17968.880476
31,091620000000,female,18,13484.093988
32,091620000000,female,20,48623.124105
33,091620000000,female,25,71253.147556
34,091620000000,female,30,135063.436146
35,091620000000,female,40,104796.766658
